<a href="https://colab.research.google.com/github/SuperCoolCucumber/CompLing/blob/main/GPT_homework10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [2]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

**Датасет со стихами Маяковского**

In [3]:
from transformers import TextDataset, DataCollatorForLanguageModeling

train_path = 'mayakovskyi.txt'

train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [4]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [5]:
trainer.train()

***** Running training *****
  Num examples = 3
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.06406107902526856, metrics={'train_runtime': 11.4528, 'train_samples_per_second': 26.195, 'train_steps_per_second': 8.732, 'total_flos': 9798451200000.0, 'train_loss': 0.06406107902526856, 'epoch': 100.0})

### ArgMax

In [6]:
text = "Я сказал что"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=1.4,
                        top_k=50,
                        max_length=200,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я сказал что мне нужен другой мужчина.. и чтобы у меня была семья. Пусть его голос будет на моём сердце
Ты и сама ответила на свой вопрос. Если ты счастлива со мной, то значит все ещё любишь. Если же ты плачешь, горюешь, жалеешь, не веришь в Бога, знай: никто другой не любил тебя.
Посадь в клетку всех голубей и беги вдаль,
увидь радугу и солнышек.
Приди на мою могилу,
накорми меня впрок.
На лугах ярмаркок
певцы дудки звон.
Радуга, дай бей челн смелым.
Грудь мужчинам, глотните
воздуха, надышавшись им.
Есть мед, на котором
никогда не гаснут льют.
Деве Марии
исполняется пению.
Сердце девы
плывёт в ладу.
Сердце девы
страшно, бъ


Началось с драмы личной, закончилось общественной. Налицо главная проблема сгенерированных текстов -- звучат вроде бы правдоподобно, но на больших дистанциях логическая связность слабая.

### Beam Search

In [9]:
out = model.generate(input_ids, do_sample=False, num_beams=20, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я сказал что-нибудь?
Выше, гордых голов гряда!
Мы разливом второго потопа
перемоем миров города.

Дней бык пег.
Медленна лет арба.
Наш бог бег.
Сердце наш барабан.




Плохо знаю Маяковского, но вроде довольно похоже.

### Temperature

In [17]:
out = model.generate(input_ids, do_sample=True, temperature=0.1, max_length=50)
generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Я сказал что-то вроде: «Я не понимаю, что происходит».

– Что происходит?

– Мы не можем понять, что происходит.

– Что происходит?

– Мы не можем понять,


relatable

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

1) Все три модели были разработаны гугловской лабораторией OpenAI.

GPT-1: [Improving Language Understanding
by Generative Pre-Training](https://www.gwern.net/docs/www/s3-us-west-2.amazonaws.com/d73fdc5ffa8627bce44dcda2fc012da638ffb158.pdf) -- 2018 год

GPT-2: [Language Models are Unsupervised Multitask Learners](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) -- 2019 год

GPT-3: [Language Models are Few-Shot Learners](https://arxiv.org/pdf/2005.14165.pdf) -- 2020 год

2) Создатели взяли самый большой корпус текстовых данных -- Common Crawl dataset. Чтобы повысить качество текстов, исследователи предприняли следующее:
- отфильтровали датасет по схожести с другими корпусами выского качества
- применили fuzzy deduplication: удаляли дубли почти похожих фрагментов
- аугментировали корпус в тренировочной выборке текстами из других высококачественных корпусов.